# Dirichlet Boundaries

The Basic Pipeline:

1) **The Marker Function**


*    Define the Marker Function that marks Boundary A

*    It takes in array  $[\vec{x}]_i$ and gives out array $[f(\vec{x})]_i$


*  $[f(\vec{x})]_i$ is an array of[True/False]$_i$ 

* True $\implies \vec{x}_j $ is in Boundary A


2) **Detecting Mesh Entities with Marker Function**

* Mesh : $ \Omega $ 
* Dimension of Boundary:  : $dim( \partial \Omega)$
* Marker function:    $[f(\vec{x})]_i$
* BoundaryA = mesh.locate_entities_boundary( $\Omega$ , dim($\partial \Omega$),  $[f(\vec{x})]_i$)


3) **Assigning the Boundary its Value**

* Determine the space of functions used for Basis : V
* determine the elements of V which are on BoundaryA :

  X=dolfinx.fem.locate_dofs_topological(V, dim($\partial \Omega$), BoundaryA)

* Define Value Function: $g(\vec{x})$

* Assign the Function on those values:

  bc = dolfinx.fem.dirichletbc(Value,X)




# The Package Setup

In [15]:
#please run twice
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/9b21f39/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [16]:
try:
    import viskex
except ImportError:
    !pip3 install "viskex@git+https://github.com/viskex/viskex.git@8895f22"
    import viskex

import ufl
import numpy as np

from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io, nls, log
import viskex

# A 3d example

In [17]:
domain = mesh.create_unit_cube(MPI.COMM_WORLD, 10, 10, 10)
#mesh is created

In [18]:
V = fem.FunctionSpace(domain, ("CG", 1))

# we define the function space where we will work

We want to give the 6 different faces of cube different values

In [19]:
# The faces are defined as x=0,x=1,y=0,y=1,z=0,z=1

def face1x(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:
    return abs(x[0] - 0.) < np.finfo(float).eps

def face2x(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:
    return abs(x[0] - 1.) < np.finfo(float).eps

def face1y(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:
    return abs(x[1] - 0.) < np.finfo(float).eps

def face2y(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:
    return abs(x[1] - 1.) < np.finfo(float).eps

def face1z(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:
    return abs(x[2] - 0.) < np.finfo(float).eps


def face2z(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:
    return abs(x[2] - 1.) < np.finfo(float).eps

  

We define the Boundary elements

In [22]:
x_facets1 = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, face1x)
y_facets1 = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, face1y)
z_facets1 = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1,face1z)
x_facets2 = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, face2x)
y_facets2 = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1, face2y)
z_facets2 = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim - 1,face2z)

# we make collections of BD elemes which we want to assign together
x_stack = np.hstack((x_facets1, x_facets2))
y_stack = np.hstack((y_facets1, y_facets2))
z_stack = np.hstack((z_facets1, z_facets2))

Now we want to apply:

 u=1 on x_stack
 
 u=2 on y_stack
 
 u=3 on z_stack

 ## Case 1: Constant Boundary Conditions

In [48]:

from petsc4py.PETSc import ScalarType
one = fem.Constant(domain, ScalarType(1))
two = fem.Constant(domain, ScalarType(2))
three = fem.Constant(domain, ScalarType(3))
# The functions have to be defined over the mesh, hence domain information is necessary



# Its a constant over the domain-mesh and with values 1,2,3


bd_x= fem.locate_dofs_topological(V,  domain.topology.dim - 1, x_stack)
bd_y= fem.locate_dofs_topological(V,  domain.topology.dim - 1, y_stack)
bd_z= fem.locate_dofs_topological(V,  domain.topology.dim - 1, z_stack)

# The boundary Degree of Freedom are identified


bc1 = fem.dirichletbc(one,bd_x,V )
bc2 = fem.dirichletbc(two,bd_y,V)
bc3 = fem.dirichletbc(three,bd_z,V )

# The boundary conditions are made


Let's try solving to see if things are sane

In [55]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(domain)

a=ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx
L=(1+2*x[0])*v*ufl.dx

u = dolfinx.fem.Function(V)

import petsc4py.PETSc


problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[bc1,bc2,bc3], u=u)
problem.solve()
u.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
viskex.dolfinx.plot_scalar_field(u, "u")

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='100...], orientation_widget=True, sizing_mode='stretch_width')

## Case 2 : Boundary is an UFL Expression

In [57]:
x = ufl.SpatialCoordinate(domain)

Expr= 1+x[0]+2*x[1]+3*x[2]
# The UFL Expression
func_exp=lambda x: eval(str(Expr))

#u_Boundary is a function in the space V
u_Boundary = fem.Function(V)

#This tells how to compute u_Boundary from u_exact
u_Boundary.interpolate(func_exp)


bd_x= fem.locate_dofs_topological(V,  domain.topology.dim - 1, x_stack)
bd_y= fem.locate_dofs_topological(V,  domain.topology.dim - 1, y_stack)
bd_z= fem.locate_dofs_topological(V,  domain.topology.dim - 1, z_stack)

# The boundary Degree of Freedom are identified


bc1 = fem.dirichletbc(u_Boundary,bd_x)
bc2 = fem.dirichletbc(u_Boundary,bd_y)
bc3 = fem.dirichletbc(u_Boundary,bd_z)

# The boundary conditions are made


Let's try solving

In [58]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(domain)

a=ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx
L=(1+2*x[0])*v*ufl.dx

u = dolfinx.fem.Function(V)

import petsc4py.PETSc


problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[bc1,bc2,bc3], u=u)
problem.solve()
u.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
viskex.dolfinx.plot_scalar_field(u, "u")

VTKRenderWindowSynchronized(vtkXOpenGLRenderWindow, color_mappers=[LinearColorMapper(id='101...], orientation_widget=True, sizing_mode='stretch_width')

# Exercise 2d geometry

In [ ]:
#  DEFINE unit square mesh 
domain=

In [ ]:
# function space is already defined
V = fem.FunctionSpace(domain, ("CG", 1))

In [ ]:
# DEFINE markers for 4 edges
# The returns are missing
def edge1x(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:


def edge2x(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:


def edge1y(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:


def edge2y(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[np.bool_]:


In [ ]:
# FILL IN THE BLANKS
x_facets1 = dolfinx.mesh.locate_entities_boundary(, domain.topology.dim - 1, )
y_facets1 = dolfinx.mesh.locate_entities_boundary(, domain.topology.dim - 1, )
x_facets2 = dolfinx.mesh.locate_entities_boundary(, domain.topology.dim - 1, )
y_facets2 = dolfinx.mesh.locate_entities_boundary(, domain.topology.dim - 1, )

# we make collections of BD elemes which we want to assign together
x_stack = np.hstack((x_facets1, x_facets2))
y_stack = np.hstack((y_facets1, y_facets2))

## CASE 1

In [ ]:
# DEFINE CONSTANT BOUNDARY CONDITIONS
# FILL IN THE BLANKS

one = fem.Constant(domain, ScalarType(1))
two = fem.Constant(domain, ScalarType(2))


bd_x= fem.locate_dofs_topological(,  domain.topology.dim - 1, )
bd_y= fem.locate_dofs_topological(,  domain.topology.dim - 1, )

bc1 = fem.dirichletbc(,bd_x,)
bc2 = fem.dirichletbc(two,bd_y,V)



In [ ]:
# Run program to check

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(domain)

a=ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx
L=(1+2*x[0])*v*ufl.dx

u = dolfinx.fem.Function(V)

import petsc4py.PETSc


problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[bc1,bc2], u=u)
problem.solve()
u.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
viskex.dolfinx.plot_scalar_field(u, "u")

## CASE 2
## VARIABLE BOUNDARY CONDITION

In [ ]:
x = ufl.SpatialCoordinate(domain)

# Fill in the Blanks

# Make expression for (x^2)*(y^2)
Expr= 
# The UFL Expression
func_exp=lambda x: eval(str(Expr))
#this converts it into a function


#u_Boundary is a function in the space V
u_Boundary = fem.Function(V)

#This tells how to compute u_Boundary from u_exact
u_Boundary.interpolate(func_exp)


bd_x= fem.locate_dofs_topological(V,  domain.topology.dim - 1, )
bd_y= fem.locate_dofs_topological(,  domain.topology.dim - 1, )

# The boundary Degree of Freedom are identified


bc1 = fem.dirichletbc(u_Boundary,bd_x)
bc2 = fem.dirichletbc(u_Boundary,bd_y)

Let's run

In [ ]:
# Run program to check

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(domain)

a=ufl.inner(ufl.grad(u),ufl.grad(v))*ufl.dx
L=(1+2*x[0])*v*ufl.dx

u = dolfinx.fem.Function(V)

import petsc4py.PETSc


problem = dolfinx.fem.petsc.LinearProblem(a, L, bcs=[bc1,bc2], u=u)
problem.solve()
u.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)
viskex.dolfinx.plot_scalar_field(u, "u")

Feel free to play around :) 